In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [ ]:
import gc
import scanpy as sc
import muon as mu
import seaborn as sns
import os
import pandas as pd
from matplotlib import pyplot as plt

os.environ['R_HOME'] = '/gpfs/bwfor/work/ws/hd_fu399-conda/conda/envs/python_R/lib/R/'
import anndata2ri
import logging

import rpy2.rinterface_lib.callbacks as rcb
import rpy2.robjects as ro

rcb.logger.setLevel(logging.ERROR)
ro.pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython

In [ ]:
%%R
library(SoupX)
library(Seurat)
library(scater)
library(scDblFinder)
library(BiocParallel)
library(MuDataSeurat)
library(DropletUtils)

In [ ]:
sample_name = "ERR7425192"

In [ ]:
metadata = pd.read_csv("../../metadata.csv", index_col = 0)

In [ ]:
raw_input = f"/home/hd/hd_hd/hd_fu399/sds-hd/sd21k006/scRNAseq/revision_natcomm/montaldo_et_al/{sample_name}/{sample_name}/outs/raw_feature_bc_matrix.h5"

In [ ]:
dataset = sc.read_10x_h5(raw_input)
dataset.var_names_make_unique()
dataset.obs_names_make_unique()

dataset.layers["raw_counts"] = dataset.X.copy()
data_tod = dataset.layers["raw_counts"].T.copy()

dataset

In [ ]:
sce = dataset.copy()

In [ ]:
%%R -i sce -o empty_drop_output
set.seed(187)
empty_drop_output <- emptyDrops(assay(sce))

In [ ]:
non_empty_barcodes = empty_drop_output[empty_drop_output["FDR"] < 0.001].index
dataset = dataset[dataset.obs.index.isin(non_empty_barcodes), :]
dataset

In [ ]:
rna_assay = dataset.copy()
sc.pp.normalize_per_cell(rna_assay)
sc.pp.log1p(rna_assay)
sc.pp.pca(rna_assay)
sc.pp.neighbors(rna_assay)
sc.tl.leiden(rna_assay, key_added="soupx_groups")

soupx_groups = rna_assay.obs["soupx_groups"]

del rna_assay

genes = dataset.var_names
cells = dataset.obs_names
data = dataset.X.T.copy()

In [ ]:
%%R -i data -i data_tod -i genes -i cells -i soupx_groups -o out 
set.seed(187)
# specify row and column names of data
rownames(data) = genes
colnames(data) = cells
# ensure correct sparse format for table of counts and table of droplets
data <- as(data, "sparseMatrix")
data_tod <- as(data_tod, "sparseMatrix")

# Generate SoupChannel Object for SoupX 
sc = SoupChannel(data_tod, data, calcSoupProfile = FALSE)

# Add extra meta data to the SoupChannel object
soupProf = data.frame(row.names = rownames(data),
                      est = rowSums(data)/sum(data),
                      counts = rowSums(data))
sc = setSoupProfile(sc, soupProf)
# Set cluster information in SoupChannel
sc = setClusters(sc, soupx_groups)

# Estimate contamination fraction
sc  = autoEstCont(sc, doPlot=FALSE, forceAccept = TRUE)
# Infer corrected table of counts and rount to integer
out = adjustCounts(sc, roundToInt = TRUE)

In [ ]:
dataset.layers["soupX_counts"] = out.T
dataset.X = dataset.layers["soupX_counts"].copy()

In [ ]:
data_mat = dataset.X.T

In [ ]:
%%R -i data_mat -o droplet_class

set.seed(187)
sce = scDblFinder(
    SingleCellExperiment(
        list(counts=data_mat),
    ) 
)
droplet_class = sce$scDblFinder.class

In [ ]:
dataset.obs["scDblFinder_class"] = droplet_class
dataset.obs.scDblFinder_class.value_counts()
dataset

In [ ]:
dataset = dataset[dataset.obs["scDblFinder_class"] == 1]

In [ ]:
# mitochondrial genes
dataset.var["mt"] = dataset.var_names.str.startswith("MT-")
# ribosomal genes
dataset.var["ribo"] = dataset.var_names.str.startswith(("RPS", "RPL"))
# hemoglobin genes.
dataset.var["hb"] = dataset.var_names.str.contains(("^HB[^(P)]"))

sc.pp.calculate_qc_metrics(dataset,
                           qc_vars=['mt', 'ribo', 'hb'],
                           percent_top=None,
                           log1p=False,
                           inplace=True)
dataset

In [ ]:
fig, ax = plt.subplots(ncols = 4, figsize = (16,4))
axis = sc.pl.violin(dataset, "n_genes_by_counts", jitter = 0.4, ax = ax[0], show = False)
axis.grid(False)
axis.set_title("Unique Genes per cell")
axis.tick_params(axis = "x", bottom =False, labelbottom =False)
axis.set_ylabel("# genes")
axis.axhline(y = 250, color = "black", label = "cutoff")
axis.legend(fontsize = 14, loc = "upper right")

axis = sc.pl.violin(dataset, "pct_counts_mt", jitter = 0.4, ax = ax[1], show = False)
axis.grid(False)
axis.set_title("Mitochondrial Fraction")
axis.tick_params(axis = "x", bottom =False, labelbottom =False)
axis.set_ylabel("% mitochondrial genes")
axis.axhline(y = 10, color = "black", label = "cutoff")
axis.legend(fontsize = 14, loc = "upper right")

axis = sc.pl.violin(dataset, "pct_counts_hb", jitter = 0.4, ax = ax[2], show = False, size =2)
axis.grid(False)
axis.set_title("Hemoglobin Fraction")
axis.tick_params(axis = "x", bottom =False, labelbottom =False)
axis.set_ylabel("% hemoglobin genes")
axis.axhline(y = 1, color = "black", label = "cutoff")
axis.legend(fontsize = 14, loc = "upper right")
#axis.set_ylim(0,5)

axis = sc.pl.violin(dataset, "pct_counts_ribo", jitter = 0.4, ax = ax[3], show = False)
axis.grid(False)
axis.set_title("Ribosomal Fraction")
axis.tick_params(axis = "x", bottom =False, labelbottom =False)
axis.set_ylabel("% ribosome genes")

plt.tight_layout()
#plt.savefig(f"{outputDir}08_gene_based_filtering.pdf", dpi = 300)
plt.show()



In [ ]:
dataset = dataset[dataset.obs.pct_counts_mt < 10, :]
dataset = dataset[dataset.obs.n_genes_by_counts > 250, :]
dataset = dataset[dataset.obs.pct_counts_hb < 1, :]
dataset

In [ ]:
for col in metadata.columns:
    dataset.obs[col] = metadata.loc[(metadata["ENA_RUN"] == sample_name) & (metadata["read_index"] == "read1"), col].to_list()[0]

In [ ]:
dataset.write(f"../../int_data/{sample_name}_qc.h5ad")